# Homework 4 - RL #

### Import packages used throughout the program ###

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.ml.linalg import Vectors
import numpy as np
import random
import time

### Homework 3 functions ###

In [2]:
def f(string):
    return Vectors.dense([float(coord) for coord in string.split(' ')])

In [3]:
def farthest_first_traversal(P, k):
    """Return a list of the k farthest points.
    
    Input:  P - a list of vectors, representing points.
            k - a scalar, number of target centroids.
    Output: S - a list of vectors, representing the centers.
    """
    idx_rnd = random.randint(0, len(P)-1)
    S = [P[idx_rnd]]
    related_center_idx = [idx_rnd for i in range(len(P))]
    dist_near_center = [Vectors.squared_distance(P[i], S[0]) for i in range(len(P))]

    for i in range(k-1):    
        new_center_idx = np.argmax(dist_near_center)

        S.append(P[new_center_idx])

        for j in range(len(P)):
            if j != new_center_idx:
                dist = Vectors.squared_distance(P[j], S[-1])
                if dist < dist_near_center[j]:
                    dist_near_center[j] = dist
                    related_center_idx[j] = new_center_idx
            else:
                dist_near_center[j] = 0
                related_center_idx[j] = new_center_idx
    return S

### Homework 4 functions ###

In [4]:
# points is a list of Vectors , k an integer
def runSequential(points, k):

    n = len(points)
    if k >= n:
        return points

    result = list()
    candidates = np.full(n , True)

    for iter in range(int(k / 2)):
        maxDist = 0.0
        maxI = 0
        maxJ = 0
        for i in range(n):
            if candidates[i] == True:
                for j in range(n):
                    d = Vectors.squared_distance(points[i], points[j])
                    if d > maxDist:
                        maxDist = d
                        maxI = i
                        maxJ = j
        result.append( points[maxI] )
        result.append( points[maxJ] )
        #print "selecting "+str(maxI)+" and "+str(maxJ)
        candidates[maxI] = False
        candidates[maxJ] = False

    if k % 2 != 0:
        s = np.random.randint(n)
        for i in range(n):
            j = (i + s) % n
            if candidates[j] == True:
                #print "selecting "+str(j)
                result.append( points[i] )
                break

    return result

input_length = 100
k = 11
d = 3
m = 10

test = list()
for i in range(input_length):
    test.append( Vectors.dense([ np.random.randint(m) for i in range(d) ]) )

#print test
print("Input of size ", str(len(test)), " with k = ", str(k))
print("Selected: ", str(len(runSequential(test, k))))

Input of size  100  with k =  11
Selected:  11


In [5]:
def runMapReduce(pointsrdd, k, numBlocks):
    
    # the RDD is already partitioned into numBlocks partitions but i need to refer to each single one
    blocks = pointsrdd.glom()
    centers = blocks.map(lambda P: farthest_first_traversal(P,k))

    return centers    

    coreset = []  # list of lists, each one containing k centers
    
    for i in range(numBlocks):
        print("Partition ", i, " contains ", len(blocks[i]))
        centers = farthest_first_traversal(blocks[i], k)  # list of vectors
        print("Centers found with farthest first traversal algorithm:\n", centers)
        coreset.append(vec for vec in centers)
    print("Coreset:\n", len(coreset))
    return runSequential(coreset, k)

### main program ###

In [6]:
# spark setup
conf = SparkConf().setAppName('HW4').setMaster('local')
sc = SparkContext(conf=conf)

In [7]:
datafile = "test-datasets/prova.txt"
numBlocks = 4
k = 2

# read file into RDD
inputrdd = sc.textFile(datafile) \
             .map(f) \
             .repartition(numBlocks) \
             .cache()
print("Number of vectors in RDD: ", inputrdd.count(), "\n")

# solve diversity maximization problem
centers = runMapReduce(inputrdd, k, numBlocks)

Number of vectors in RDD:  11 



In [8]:
 print("Type: ", type(centers))

Type:  <class 'pyspark.rdd.PipelinedRDD'> 



In [9]:
 print("Type: ", len(centers))

TypeError: object of type 'PipelinedRDD' has no len()

In [10]:
 print("Type: ", centers.count())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 3.0 failed 1 times, most recent failure: Lost task 1.0 in stage 3.0 (TID 6, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\opt\spark\spark-2.3.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 229, in main
  File "C:\opt\spark\spark-2.3.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 224, in process
  File "C:\Users\Riccardo\Anaconda3\lib\site-packages\pyspark\rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\Riccardo\Anaconda3\lib\site-packages\pyspark\rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\Riccardo\Anaconda3\lib\site-packages\pyspark\rdd.py", line 2438, in pipeline_func
    retu

In [11]:
 print("Type: ", centers.collect())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 5.0 failed 1 times, most recent failure: Lost task 1.0 in stage 5.0 (TID 9, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\opt\spark\spark-2.3.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 229, in main
  File "C:\opt\spark\spark-2.3.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 224, in process
  File "C:\opt\spark\spark-2.3.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-5-ec593e2b1a29>", line 5, in <lambda>
  File "<ipython-input-3-aa05e30726fb>", line 8, in farthest_first_traversal
  File "C:\Users\Riccardo\Anaconda3\lib\random.py", line 221, in randint
    return self.randrange(a